## Reading and writing from files

The `open` function is the standard way to gain access to a file on your filesystem.  Files can be opened for reading only, writing, or both reading and writing.  By default files are opened in read only mode.  The primary argument to the `open` function is a string giving the path and name of the file you want to open:

In [15]:
### open function and file objects

## You can download `covid-ref.fsa` from the course repository at 
##    https://github.com/Bio724D/Bio724D_2023_2024/tree/main/data
#
## This is a FASTA file of the Sars-Cov-2 reference genome
## See: https://www.ncbi.nlm.nih.gov/nuccore/NC_045512
##
## the default is to open the file in read-only mode
f = open("covid-ref.fsa")  

The simplest way to manipulate a file is simply to read all the information from it, return the data in the file as a string:

In [16]:
### reading everything from a file as a single string
s = f.read()
type(s)

str

Once you've read what you need from the file it's good practice to close it (failing to close a file can lead to a memory leak in some contexts, but it's usually not a problem in an interactive environment like Jupyter notebooks).

In [17]:
f.close()

An alternate way to read a file is within a `with` statement as illustrated below.  The advantage of the `with` statement is it insures the file is closed (i.e. you don't need to explicitly call the `close` method) at the end of the `with` block.

In [18]:
with open("covid-ref.fsa") as f:
    s = f.read()

Once we've read the file into a string we can apply all the standard string methods and operators to it:

In [19]:
len(s)  ## how many characters were in the file?

30429

In [20]:
s[:50]  ## first 50 chars in the file

'>NC_045512.2 Severe acute respiratory syndrome cor'

In [21]:
s[-50:]  ## last 50 characters

'CTTAGGAGAATGACAAAAAAAAAAAAAAAAAAAA\nAAAAAAAAAAAAA\n\n'

## Reading a file by lines

Sometimes it's more convenient or more efficient to get the information in the file in terms of lines.  The `readlines` method associated with file object read's all the lines at once in a list:

In [22]:
## return a list of the lines in the file
with open("./covid-ref.fsa") as f:
    lines = f.readlines()

In [23]:
type(lines)

list

In [24]:
len(lines)

430

In [25]:
# the first line is a "header" line (see explanation of FASTA format below)
# header lines start with a right bracket (>)
lines[0]

'>NC_045512.2 Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome\n'

In [26]:
# subsequent lines are the actual sequence data
lines[1]

'ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAA\n'

Notice the newline character (`\n`) at the end of each line.

## Reading a file one line at a time

The `readlines` function illustrated above reads all the lines at once. That's works well if your file has a modest number of lines, but for a file with millions of lines (or very long  lines)  `readlines` might exhaust the memory of your computer.  One way to work around this is to process files line by line, reading only one line at a time.  This can be done with a for loop applied directly to the file object:

In [27]:
with open("./covid-ref.fsa") as f:
    nchars = []  # create a list to hold characters counts per line
    for line in f:
        nchars.append(len(line)) 

nchars[:10]  # the first 10 counts

[97, 71, 71, 71, 71, 71, 71, 71, 71, 71]

For such a simple computation we'd typically use a list comprehension which you can think of as a compact version of a for loop (more on list comprehensions in a later class)

In [28]:
with open("./covid-ref.fsa") as f:
    nchars = [len(line) for line in f]

nchars[:10] 

[97, 71, 71, 71, 71, 71, 71, 71, 71, 71]

## Iterating through a file, filtering and concatenating lines

Here we illustrate the process of iterating through the lines of a file, doing some simple filtering, and concatenating lines into a single string.

In [29]:
# iterate through the lines of 
seq = ""
with open("./covid-ref.fsa") as f:
    for line in f:
        if line[0] == ">":  # if the line starts with right bracket
            continue        # skip it (i.e. go to next iteration of for loop)
        if len(line.strip()):    # if the line is not empty
            seq += line.strip()  # add it to our seq string object
            # NOTE: strip removes newlines and other whitespace at beginning/end of lines


In [30]:
# total length of COVID reference genome nucleotide sequence
# does not include the header line which we filtered out in our for loop above
len(seq)

29903

In [31]:
# first 100 characters in this sequence
seq[:100]

'ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC'

## Extracting codons  using string slicing

Since the string we're working with represents the Sars-Cov-2 genome, let's extract a subsequence of interest that represents the gene that encodes the Spike protein. Once we've done so we'll generate  codons from that subsequence and translate them to their corresponding amino acids.

The spike protein coordinates from NCBI are given as: 21563..25384 but these are 1-indexed and inclusive of start/end coordinates. To extract the corresponding sequence from our Python string we need to convert these to 0-indexed coordinates and remember that Python indexing is up to but not including end index

In [32]:
# Spike protein coords from NCBI are given as: 21563..25384
# Note conversion to 0-indexing and non-inclusive end index

spike = seq[21562:25384]

In [33]:
# length of sequence we extracted
len(spike)

3822

In [34]:
# first 10 nucleotides
spike[:10]

'ATGTTTGTTT'

Next we'll extract the nucleotide triplets that represent the codons of the spike protein. This task is simple in this case because there are no introns to consider and the gene is encoded in the same strand orientation as the data is provided to us (not always the case)

In [35]:
# create a list of the start positions for each codon
codon_starts = range(0, len(spike), 3)

# we wrap this in a call to list() to see what it looks like
# because range() returns an iterator
list(codon_starts)[:10]  

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]

In [36]:
# extract codon subsequences by indexing with codon starts and slicing three characters
spike_codons = [spike[i:i+3] for i in codon_starts]
spike_codons[:10]

['ATG', 'TTT', 'GTT', 'TTT', 'CTT', 'GTT', 'TTA', 'TTG', 'CCA', 'CTA']

Now that we have codons we can explore setting up a dictionary to handle the translation. We're going to pull the genetic code information from the standard codon table given by NCBI (see below), which we'll arrange as a dictionary.  This dictionary maps from codons (the keys) to single-letter representations of amino acids.

In [37]:
# NCBI provides the described genetic codes here
#   https://www.ncbi.nlm.nih.gov/Taxonomy/Utils/wprintgc.cgi#SG1
# Below is the standard code
# I did a simple cut and paste of the data from the NCBI website to create the strings below

Base1  = "TTTTTTTTTTTTTTTTCCCCCCCCCCCCCCCCAAAAAAAAAAAAAAAAGGGGGGGGGGGGGGGG"
Base2  = "TTTTCCCCAAAAGGGGTTTTCCCCAAAAGGGGTTTTCCCCAAAAGGGGTTTTCCCCAAAAGGGG"
Base3  = "TCAGTCAGTCAGTCAGTCAGTCAGTCAGTCAGTCAGTCAGTCAGTCAGTCAGTCAGTCAGTCAG"
AAs    = "FFLLSSSSYY**CC*WLLLLPPPPHHQQRRRRIIIMTTTTNNKKSSRRVVVVAAAADDEEGGGG"

## for example "TTT" (first column) translates to "F" (phenylalanine)
## "GGG" (last column) translate to "G" (glycine), "TAG" to "*" (stop codon), etc

In [38]:
# we can turn those strings into a dictionary as below

codon2AA = dict() # empty dictionary

for i in range(len(AAs)):    # like R's 1...leng
    codon = Base1[i] + Base2[i] + Base3[i]  # create codon string
    codon2AA[codon] = AAs[i]  # add entry mapping codong string to corresponding AA


In [39]:
## test our dict
codon2AA["TTT"], codon2AA["GGG"], codon2AA["TAG"]

('F', 'G', '*')

Having setup this dictionary mapping codons to corresponding amino acids we can "translate" the codons using simple dictionary lookup. Here we do this in a for loop, but a list comprehension would be more idiomatic.

In [40]:
spike_AAs = []

for codon in spike_codons:
    spike_AAs.append(codon2AA[codon])

spike_AAs[:10]

['M', 'F', 'V', 'F', 'L', 'V', 'L', 'L', 'P', 'L']

In [41]:
# if you have a list of strings and want to join them into a single string
# the join() method is useful
spike_protein = "".join(spike_AAs)  # what happens if you write "!".join(spike_AAs) instead?
spike_protein[:100] # first 100 AAs


'MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNI'

In [42]:
spike_protein[-100:] # last 100 AAs

'SVVNIQKEIDRLNEVAKNLNESLIDLQELGKYEQYIKWPWYIWLGFIAGLIAIVMVTIMLCCMTSCCSCLKGCCSCGSCCKFDEDDSEPVLKGVKLHYT*'

# The FASTA file format for sequence data

The FASTA file format is the most commonly used file format used to represent nucleotide and protein sequence data.  Wikipedia has a good [overview of the FASTA format](https://en.wikipedia.org/wiki/FASTA_format).  

Summary of FASTA format:
 
 * Each file can hold one or more sequence records
 
 * The beginning of each record is delimited by a line called a header, which has a `>` character at the beginning, followed by the name associated with that record (and an optional description). For example `>seq1 Involved in...` would indicate the beginning of a record with the name `seq1` and the description "Involved in...".
 
 * On or more sequence lines follow header lines.  These lines are usually wrapped to have length <=80 characters but this is not required.



Below is a simple function that will parse a FASTA file, returning each record as an element in a Python dictionary with the first word in the header line of each record as the key. 

This implementation isn't particularly robust or optimal, but illustrates some key aspects of flow-control in Python and parsing non-tabular data.


In [43]:
def parse_FASTA(fname):
    record_dict = {}
    f = open(fname, 'r')
    
    recname = ""           # will hold names of records
    seq = ""               # will hold seq strings
    active_record = False  # indicates whether we are currently working on building a record
    
    for line in f.readlines():
        
        line = line.strip()  # strip any whitespace at beginning/end of line
        
        if line == "":       # empty line
            continue         # go to next iteration of for loop
            
 
        if line[0] == ">":                 # are we dealing with a new record?
            if active_record:              # did we already have an active record?
                record_dict[recname] = seq # if so, add to old active record to the dict so we can
                                           # begin a new one 
            
            recname = line[1:].split()[0]  # name of new record
            seq = ""                       # reset variable holding the string
            active_record = True           # set flag to indicate we now have an active record
            continue                       # go to the next iteration of for loop, as there's nothing else to do
        
        seq += line
        
    if active_record:               # if we've exhausted all the lines, we might still have an active record
        record_dict[recname] = seq  # if so, add it to the dict

    f.close() # remember to close the file!

    return record_dict
        
            

To test our `parse_FASTA` function download the [`Spike-protein-aligned.fasta`](https://github.com/Bio724D/Bio724D_2023_2024/tree/main/data/Spike-protein-aligned.fasta) file  to your computer and modify the paths below as necessary to load and parse the sequence records contained in that file.

In [44]:
# load the data
recs = parse_FASTA("./Spike-protein-aligned.fasta")

In [45]:
# examine the type of object we got back
type(recs)

dict

In [46]:
# how many records are there
len(recs)

7

In [47]:
# what are the keys of the dictionary of recrods
list(recs.keys())  

['KF367457.1',
 'AY278488.2',
 'NC_004718.3',
 'MG772933.1',
 'MT040333.1',
 'MN908947.3',
 'MN996532.1']

In [48]:
# get a specific record, and the first 80 AAs
recs["MN996532.1"][:80]

'M-FVFLVL-LPLVSS----QCVNLTTRTQLPPAYTN--SSTRGVYYPDKVFRSSVLHLTQDLFLPFFSNVTWFHAIHVSG'

In [49]:
# print the first 25 positions in the alignment for all the records
for key in recs.keys():
    print(recs[key][:25])

MKLLVLVF-ATLVSSYTIEKCLDFD
M-FIFLLF-LTLTSGSDLDRCTTFD
M-FIFLLF-LTLTSGSDLDRCTTFD
M-LFFLFLQFALVNS----QCVNLT
M-FVFLFV-LPLVSS----QCVNLT
M-FVFLVL-LPLVSS----QCVNLT
M-FVFLVL-LPLVSS----QCVNLT
